In [2]:
import sys
import copy
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset, Subset, random_split
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
else:
    print("CUDA is not available. Using CPU.")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def load_dataset2():
    # Data preprocessing
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    # Load the full MNIST training dataset
    full_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

    # 20,000 samples were randomly selected
    subset_indices = torch.randperm(len(full_dataset))[:20000]
    subset_dataset = Subset(full_dataset, subset_indices)

    #  Divide 20,000 samples into 5,000 training sets, 5,000 validation sets, and 10,000 test sets
    train_set, val_set, test_set = random_split(subset_dataset, [5000, 5000, 10000])

    # Scramble the labeling of 2,500 samples in the training set
    rand_indices = torch.randperm(len(train_set))[:2500]
    for idx in rand_indices:
        # A new tag is randomly generated
        new_label = torch.randint(0, 10, (1,)).item()
        train_set.dataset.dataset.targets[subset_indices[train_set.indices[idx]]] = new_label

    # Create a data loader
    trainloader = DataLoader(train_set, batch_size=64, shuffle=True)
    valloader = DataLoader(val_set, batch_size=64, shuffle=True)
    testloader = DataLoader(test_set, batch_size=64, shuffle=True)

    return trainloader, valloader, testloader

def test(net, testloader):
    # Test the network
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            # forecast
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total if total > 0 else 0
    return accuracy

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(28*28, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.fc(x)
        return x

def train(lower_lr, upper_lr):
    # Define the neural network
    net_old = SimpleNet().to(device)

    # Define the loss function and optimizer
    lower_optimizer = optim.SGD(net_old.parameters(), lr=lower_lr, momentum=0.9)
    upper_optimizer = optim.Adam(net_old.parameters(), lr=upper_lr)

    trainloader, valloader, testloader = load_dataset2()

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    def lower_function(output, label, la):
        crossentropy = nn.CrossEntropyLoss()
        loss = crossentropy(output, label) * la
        return loss

    def upper_function(output, label):
        crossentropy = nn.CrossEntropyLoss()
        loss = crossentropy(output, label) + 0.01 * (torch.norm(net_old.fc.weight) + torch.norm(net_old.fc.bias))
        return loss

    def inner_loop(trainloader, net, la):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            lower_optimizer.zero_grad()

            outputs = net(inputs)
            loss = lower_function(outputs, labels, la[i])
            loss.backward()
            lower_optimizer.step()

            running_loss += loss.item()
        return running_loss

    def outer_loop(trainloader, net, la):
        upper_optimizer.zero_grad()

        upper_loss = 0.0
        for i, data in enumerate(valloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = net(inputs)
            loss = upper_function(outputs, labels)

            upper_loss += loss

        upper_loss.backward()
        upper_optimizer.step()

        return upper_loss

    T = 10
    la = torch.rand([5000, 1], requires_grad=True).to(device)
    
    for epoch in range(T):
        lower_loss = inner_loop(trainloader, net_old, la)
        train_losses.append(lower_loss)

        if epoch % 10 == 9:
            print(f'[Epoch {epoch + 1}] lower_loss: {lower_loss / 200:.3f}')

        upper_loss = outer_loop(trainloader, net_old, la)
        val_losses.append(upper_loss)

        train_accuracy = test(net_old, trainloader)
        val_accuracy = test(net_old, valloader)

        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        
    return net_old, train_losses, val_losses, train_accuracies, val_accuracies

if __name__ == "__main__":
    # Hyperparameters for hyperparameter optimization
    lower_lr_candidates = [0.01, 0.001, 0.0001]
    upper_lr_candidates = [0.01, 0.001, 0.0001]

    best_accuracy = 0.0
    best_lower_lr = 0.0
    best_upper_lr = 0.0

    for lower_lr in lower_lr_candidates:
        for upper_lr in upper_lr_candidates:
            print(f"Training with lower_lr={lower_lr}, upper_lr={upper_lr}")
            net_trained, train_losses, val_losses, train_accuracies, val_accuracies = train(lower_lr, upper_lr)
            _, _, testloader = load_dataset2()
            accuracy = test(net_trained, testloader)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_lower_lr = lower_lr
                best_upper_lr = upper_lr

    print(f"Best accuracy: {best_accuracy}%, Best lower_lr: {best_lower_lr}, Best upper_lr: {best_upper_lr}")

    


CUDA is not available. Using CPU.
Training with lower_lr=0.01, upper_lr=0.01
[Epoch 10] lower_loss: 0.382
Training with lower_lr=0.01, upper_lr=0.001
[Epoch 10] lower_loss: 0.360
Training with lower_lr=0.01, upper_lr=0.0001
[Epoch 10] lower_loss: 0.406
Training with lower_lr=0.001, upper_lr=0.01
[Epoch 10] lower_loss: 0.486
Training with lower_lr=0.001, upper_lr=0.001
[Epoch 10] lower_loss: 0.345
Training with lower_lr=0.001, upper_lr=0.0001
[Epoch 10] lower_loss: 0.372
Training with lower_lr=0.0001, upper_lr=0.01
[Epoch 10] lower_loss: 0.592
Training with lower_lr=0.0001, upper_lr=0.001
[Epoch 10] lower_loss: 0.386
Training with lower_lr=0.0001, upper_lr=0.0001
[Epoch 10] lower_loss: 0.398
Best accuracy: 83.08%, Best lower_lr: 0.0001, Best upper_lr: 0.01


In [3]:
# Plotting
    epochs = range(1, 21)  # Assuming 20 epochs
    plt.figure(figsize=(12, 8))

    # Plot Training and Validation Loss
    plt.subplot(2, 2, 1)
    plt.plot(epochs, train_losses, label='Training Loss')
    plt.plot(epochs, val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

IndentationError: unexpected indent (1371478360.py, line 2)

In [4]:
# Plotting

    # Plot Training and Validation Accuracy
    plt.subplot(2, 2, 2)
    plt.plot(epochs, train_accuracies, label='Training Accuracy')
    plt.plot(epochs, val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

IndentationError: unexpected indent (1744056592.py, line 4)

In [5]:
 # Plot Best Hyperparameters
    plt.subplot(2, 2, 3)
    plt.scatter(best_lower_lr, best_upper_lr, color='red', label='Best Hyperparameters')
    plt.xlabel('Lower Learning Rate')
    plt.ylabel('Upper Learning Rate')
    plt.title('Best Hyperparameters')

    plt.tight_layout()
    plt.show()

IndentationError: unexpected indent (1305686293.py, line 2)